# Bölüm Projesi: Sentiment Metin Sınıflandırma Modeli

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
from textblob import TextBlob

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
import pandas as pd
data = pd.read_csv("train.tsv", sep="\t")
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


veri etiketli (sentiment skorları ifade edilmiş) fakat ben bu etiketleri 2 sınıfa indirgeyeceğim yani yeniden metin üzerinden sentiment skorlarını oluşturacağım. yorumları pozitif yorum ve negatif yorum olacak şekilde 2 başlıkta değerlendirmeye çalışacağım. yani binary classification problemine dönüşecek. bu veri setinde negatif yorumlar 0 ve 1, pozitif yorumlar 3 ve 4 ile temsil edilmiştir.

In [ ]:
# 0 ve 1 negatif olarak değiştirildi.
# inplace = True işlemin kalıcı olmasını sağladı.
data["Sentiment"].replace(0, value = "negatif", inplace = True)
data["Sentiment"].replace(1, value = "negatif", inplace = True)

In [ ]:
data["Sentiment"].replace(3, value = "pozitif", inplace = True)
data["Sentiment"].replace(4, value = "pozitif", inplace = True)

In [ ]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [ ]:
# 2 sınıfını kaldırıyorum
data = data[(data.Sentiment == "negatif") | (data.Sentiment == "pozitif")]

In [ ]:
data.groupby("Sentiment").count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
negatif,34345,34345,34345
pozitif,42133,42133,42133


In [ ]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
21,22,1,good for the goose,pozitif
22,23,1,good,pozitif
33,34,1,"the gander , some of which occasionally amuses...",negatif
46,47,1,amuses,pozitif


## Veri Ön İşleme

In [ ]:
# sadece lazım olan değişkenlerle yeni bir df oluşturdum.
df = pd.DataFrame()
df["text"] = data["Phrase"]
df["label"] = data["Sentiment"]

In [ ]:
df.head()

,text,label
0,A series of escapades demonstrating the adage ...,negatif
21,good for the goose,pozitif
22,good,pozitif
33,"the gander , some of which occasionally amuses...",negatif
46,amuses,pozitif


In [ ]:
# küçük dönüşümü
df['text'] = df['text'].apply(lambda x: " ".join(word.lower() for word in x.split()))
#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')
#sayılar
df['text'] = df['text'].str.replace('\d','')

#stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(word for word in x.split() if word not in sw))

#seyreklerin silinmesi
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(word for word in x.split() if word not in sil))

#lemmi
from textblob import Word
nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join(Word(e).lemmatize() for e in x.split()))


<ipython-input-13-f7db07a55d12>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')
<ipython-input-13-f7db07a55d12>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d','')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Değişken Mühendisliği

Elimizdeki metinleri nümerik olarak temsil edebilmek adına bu metinlerden bazı nümerik bilgiler çıkarmamız gereklidir. bir metnin içerisinden özellik çıkaracağız (değişken oluşturacağız)

Kullanılan Yöntemler:
*   Count Vectors: her bir satır, bir yorumu ya da dökümanı temsil eder. Her bir sütun ise her bir dökümandaki kelimeleri ifade eder. kesişimlerinde yer alan ifadeler ise dökümanlarda geçme sıklığını ifade eder.

*   TF-IDF Vectors (words, characters, n-grams):
> * TF(t) = (Bir t teriminin bir dökümanda gözlenme frekansı) / (dökümandaki toplam terim sayısı)
> * IDF(t) = log_e(Toplam döküman sayısı / içinde t terimi olan belge sayısı)

*   Word Embeddings: kelime ve dökümanları yoğunluk vektörü gösterimi ile temsil etmenin bir yoludur. yani bir kelimenin vektör uzayındaki pozisyonu metinden öğreniliyor ve bu kelimenin konumu kendini çevreleyen kelimelere bağlı olduğundan dolayı burada kelime ve dökümanlar için bir yoğunluk vektörü oluşmuş olur. Embeddings, kullandığımız veri seti kullanılarak eğitilebilir ya da önceden eğitilmiş Word Embeddings'ler kullanılarak bu işlem gerçekleştirilir.

### Test-Train

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df["text"],
                                                                    df["label"],
                                                                    random_state = 1)

In [ ]:
train_x.head()

118788    present fascinating glimpse urban life class w...
89514     hey everybody wanna watch movie guy dressed ch...
86857                               incredible number story
140626                         ultimately disappoint action
153243                                              unified
Name: text, dtype: object

In [ ]:
# bağımlı değişkene (y) bir dönüştürme işlemi yapacağım
# 1 ve 0 olarak değiştireceğim
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [ ]:
train_y[:5]

array([1, 0, 1, 0, 1])

In [ ]:
test_y[:5]

array([1, 0, 1, 0, 0])

### Count Vectors

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [ ]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [ ]:
vectorizer.get_feature_names_out()[:5]

array(['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned'], dtype=object)

In [ ]:
x_train_count.toarray()[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### TF-IDF

#### word level tf-idf

In [ ]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer()

In [ ]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [ ]:
tf_idf_word_vectorizer.get_feature_names_out()[:5]

array(['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned'], dtype=object)

In [ ]:
x_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

count'dan farkı ise göreceli etkilerdir yani döküman ve veri seti bazında görece etkileri göz önünde bulundurarak bu değerleri oluşturmuş olur.

#### n-gram level tf-idf

kelimelerin birbirleriyle kombinasyonu

In [ ]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(ngram_range=(2, 3))

In [ ]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

#### characters level tf-idf

In [ ]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [ ]:
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

count, word, ngram ve character ile feature'ları oluşturmuş olduk.

## Makine Öğrenmesi ile Sentiment Sınıflandırması

bir yorum geldiğinde bu yorumun pozitif mi yoksa negatif mi olduğunu tahmin edecek bir model kurmak

her model için 4 farklı feature enginnering yöntemiyle deneme yapacağım

### Lojistik Regresyon

In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Count Vectors Doğruluk Oranı: 0.8368200836820083


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Word-Level TF-IDF Doğruluk Oranı: 0.8331589958158995


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.748326359832636


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_chars,
                                           test_y,
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

CHARLEVEL Doğruluk Oranı: 0.7811715481171548


* ***Count Vectors Doğruluk Oranı***: 0.8368200836820083
* ***Word-Level TF-IDF Doğruluk Oranı***: 0.8331589958158995
* ***N-GRAM TF-IDF Doğruluk Oranı***: 0.748326359832636
* ***CHARLEVEL Doğruluk Oranı***: 0.7811715481171548

En Yüksek doğruluk oranı, **Count Vectors** yöntemiyle bulundu. Count Vectors ve Word-Level daha başarılıdır.

### Naive Bayes

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(nb_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8332112970711296


In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.835041841004184


In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(nb_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7685146443514643


In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(nb_model,
                                           x_test_tf_idf_chars,
                                           test_y,
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7557008368200837


* Count Vectors Doğruluk Oranı: 0.8332112970711296
* Word-Level TF-IDF Doğruluk Oranı: 0.835041841004184
* N-GRAM TF-IDF Doğruluk Oranı: 0.7685146443514643
* CHARLEVEL Doğruluk Oranı: 0.7557008368200837

Count Vectors ve Word-Level daha başarılıdır.

### Random Forests

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8207635983263598


In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(rf_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(rf_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(rf_model,
                                           x_test_tf_idf_chars,
                                           test_y,
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

### XGBoost

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.7153242677824267


In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(xgb_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(xgb_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(xgb_model,
                                           x_test_tf_idf_chars,
                                           test_y,
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

## Tahmin

In [ ]:
# yeni yorumların pozitif mi negatif mi olduğunu
# lojistik regresyon ve count vectors yöntemi ile tahmin edilecek
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Count Vectors Doğruluk Oranı: 0.8368200836820083


In [ ]:
# tahmin edeceğimiz yorumu dönüştürelim
yeni_yorum = pd.Series("this film is very nice and good i like it")
yeni_yorum2 = pd.Series("no not good look at that shit very bad")

In [ ]:
v = CountVectorizer()
v.fit(train_x)

CountVectorizer()

In [ ]:
yeni_yorum = v.transform(yeni_yorum)
yeni_yorum2 = v.transform(yeni_yorum2)

In [ ]:
# yeni_yorum için tahmin
loj_model.predict(yeni_yorum)
# pozitif sınıf 1

array([1])

In [ ]:
# yeni_yorum2 için tahmin
loj_model.predict(yeni_yorum2)
# negatif sınıf 0

array([0])

* %50 kötü bir oran rastgele yapılmış olsaydı yine %50 dönderdi.
* en sık kötü yorumlar olduğu için binary'e çevirdiğimizde 0 kötü yorumlar için atandı.